# Gemini API: 嵌入 REST 快速入門

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/quickstarts/rest/Embeddings_REST.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google colab 中運作</a>
  </td>
</table>


此筆記提供快速範例程式碼，說明如何開始使用 `curl` 產生嵌入。

你可在 Google Colab 中執行，或將 `curl` 指令複製/貼到終端機中。

要執行此筆記，你的 API 金鑰必須儲存在名為 GOOGLE_API_KEY 的 Colab 密碼中。如果你是在其他環境中執行，可以將金鑰儲存在環境變數中。請參閱 [身分驗證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) 以瞭解更多資訊。


In [54]:
import os
from google.colab import userdata

In [55]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## 嵌入內容

使用 `models/embedding-001` 模型呼叫 `embed_content` 方法來產生文字嵌入：


In [56]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/embedding-001:embedContent?key=$GOOGLE_API_KEY \
-H 'Content-Type: application/json' \
-d '{"model": "models/embedding-001",
    "content": {
    "parts":[{
      "text": "Hello world"}]}, }' 2> /dev/null | head

{
  "embedding": {
    "values": [
      0.04703258,
      -0.040190056,
      -0.029026963,
      -0.026809642,
      0.018920582,
      -8.3654784e-05,
      0.031116402,


# 批次內嵌內容

你可以在一次 API 呼叫中嵌入多個提示字串的清單，以提高效率。


In [3]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/embedding-001:batchEmbedContents?key=$GOOGLE_API_KEY \
-H 'Content-Type: application/json' \
-d '{"requests": [{
      "model": "models/embedding-001",
      "content": {
      "parts":[{
        "text": "What is the meaning of life?"}]}, },
      {
      "model": "models/embedding-001",
      "content": {
      "parts":[{
        "text": "How much wood would a woodchuck chuck?"}]}, },
      {
      "model": "models/embedding-001",
      "content": {
      "parts":[{
        "text": "How does the brain work?"}]}, }, ]}' 2> /dev/null | grep -C 5 values

{
  "embeddings": [
    {
      "values": [
        -0.0002620658,
        -0.05592018,
        -0.012463195,
        -0.020672262,
        0.0076786764,
--
        0.024198001,
        0.011966026
      ]
    },
    {
      "values": [
        -0.0151748555,
        -0.050790474,
        -0.032357067,
        -0.05806994,
        0.03587488,
--
        0.014179298,
        0.032972813
      ]
    },
    {
      "values": [
        0.025271073,
        -0.064161226,
        -0.025818137,
        -0.00611377,
        -0.011130964,


## 使用 `task_type` 提供提示，說明模型將如何使用嵌入

讓我們了解 `embed_content` 方法會取得哪些參數。共有四個：

* `model`: 必填。必須為 `models/embedding-001`。
* `content`: 必填。你想要內嵌的內容。
* `task_type`: 選填。將使用嵌入項目的任務類型。請參閱下方可能的數值。
* `title`: 給定的文字是正在搜尋的語料文件。選擇性地使用 `title` 參數設定文件的標題。僅能在 `task_type` 為 `RETRIEVAL_DOCUMENT` 時設定。

`task_type` 是選擇性參數，會提供提示給 API，說明你打算如何於應用程式中使用嵌入。

系統會接受以下任務類型參數：

* `TASK_TYPE_UNSPECIFIED`: 若你未設定數值，它會預設為 `retrieval_query`。
* `RETRIEVAL_QUERY`: 給定的文字是搜尋/擷取設定的查詢。
* `RETRIEVAL_DOCUMENT`: 給定的文字是正在搜尋語料庫的文件。
* `SEMANTIC_SIMILARITY`: 給定的文字會用於語意文本相似度 (STS)。
* `CLASSIFICATION`: 給定的文字會被分類。
* `CLUSTERING`: 嵌入將會用於分群。


In [58]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/embedding-001:embedContent?key=$GOOGLE_API_KEY \
-H 'Content-Type: application/json' \
-d '{"model": "models/embedding-001",
    "content": {
    "parts":[{
      "text": "Hello world"}]},
    "task_type": "RETRIEVAL_DOCUMENT",
    "title": "My title"}' 2> /dev/null | head

{
  "embedding": {
    "values": [
      0.060187872,
      -0.031515103,
      -0.03244149,
      -0.019341845,
      0.057285223,
      0.037159503,
      0.035636507,


## 了解更多資訊

*   請參閱 [REST API 參考手冊](https://ai.google.dev/api/rest) 了解更多資訊。
*   在實戰指南中探索更多範例。
